# Chunking Strategies III: LLM-based chunking

In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken
from langchain_core.documents import Document
from IPython.display import display, Markdown
from openai import OpenAI
from src import utils

# Params

In [44]:
TOKENIZER_NAME ="cl100k_base"
EMB_MODEL="text-embedding-3-small"
LLM = "gpt-4.1-mini-2025-04-14"  # 1M token context, better than gpt-4o in structured output

[Using GPT-5](https://platform.openai.com/docs/guides/latest-model)

gpt5-main-mini no está disponible aun en API

In [45]:
path_data = Path() / "data"
path_input = path_data / "interim"
path_output = path_data / "processed" / "chunks"  # Store chunks by experiment

path_output.mkdir(exist_ok=True, parents=True)

# Environment Variables

In [46]:
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [47]:
encoding = tiktoken.get_encoding(TOKENIZER_NAME)

# Clients

In [48]:
client_openai= OpenAI(api_key=OPENAI_API_KEY)

# Load Data

In [ ]:

dc_chunks = utils.load_documents(path_output / "pages" / "Althera.json")

corpus_doc = Document(
    page_content="\n\n".join([doc.page_content for doc in dc_chunks[:4]]),
    metadata={"source": "Althera"}
)

In [50]:
stats_doc = utils.corpus_stats(encoding, [corpus_doc])
stats_doc.describe()

count       1.0
mean     2162.0
std         NaN
min      2162.0
25%      2162.0
50%      2162.0
75%      2162.0
max      2162.0
dtype: float64

In [51]:
[1,2,3,4][-2:]

[3, 4]

In [52]:
corpus_doc.page_content[-100:]

' contribuyendo a su estabilidad a largo plazo.\n![img-0.jpeg](img-0.jpeg)\n\nFigure 1: Ocaso en Althéra'

# LLM Chunking (DIY)

Input to an LLM a text based document

In [53]:
from typing import  List


from enum import Enum
from typing import List
from pydantic import BaseModel, Field, constr


class ChunkReason(str, Enum):
    section = "section"
    semantic = "semantic"
    other = "other"


class Chunks(BaseModel):
    text: constr(strip_whitespace=True, min_length=1) = Field(
        ..., description="Exactly the chunked text from the document"
    )
    summary: constr(strip_whitespace=True, min_length=1) = Field(
        ..., description="A concise summary of the extracted text"
    )
    reason_of_chunk: ChunkReason = Field(
        ..., description="The reason why this chunk was extracted."
    )
    sections_found: List[str] = Field(
        default_factory=list,
        description="List of section headers found in this chunk. Can be an list empty.",
    )

class DocumentChunked(BaseModel):
    chunks: List[Chunks] = Field(..., description="List of chunks extracted")


response = client_openai.responses.parse(
    model=LLM,
    input=[
        {"role": "system", "content": "Your task is to split a markdown document text information into chunks.\
         You should follow these roules:\
         Each chunk should be splitted base on natural text separators, such as sentences or paragraphs boundaries.\
         Each chunk ideally should have a self-contained topic.\
         Markdown headers can be missleading, so infer if they are properly placed and use them to split only if they are convenient\
         Aim at a minimum chunk size of 128 tokens and a maximun chunk size of 1024.\
         Of course, you MUST REPRODUCE EXACTLY the same text seen in the input, do not change nothing.\
         "},
        {
            "role": "user",
            "content": f"Text in markdown to chunk: \n\n{corpus_doc.page_content}",
        },
    ],
    text_format=DocumentChunked,
)

chunks = response.output_parsed



In [54]:
chunks = response.output_parsed

len(chunks.chunks)

10

In [55]:
Markdown(chunks.chunks[0].text)

# Un nuevo y fascinante vecino: Althéra 

## Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

In [56]:
Markdown(chunks.chunks[-1].text)

### 3.2 Masas, radios y luminosidades comparadas

El sistema binario Althéra está formado por dos estrellas de características bien contrastadas: Althéra A, más masiva y luminosay Althéra B, más pequeña y fría. Aunque la luminosidad de Althéra A domina el aporte energético global, la presencia de Althéra B, O, actúa como un regulador térmico que suaviza las variaciones extremas de temperatura en la zona habitable, contribuyendo a su estabilidad a largo plazo.
![img-0.jpeg](img-0.jpeg)

Figure 1: Ocaso en Althéra

In [57]:
for i, chunk in enumerate(chunks.chunks):
    print(f"chunk: {i} ----------------")
    print(f"{chunk.reason_of_chunk=}")
    print(f"{chunk.summary=}")
    print(f"{chunk.sections_found=}")
    print("Text:")
    display(Markdown(chunk.text))

chunk: 0 ----------------
chunk.reason_of_chunk=<ChunkReason.section: 'section'>
chunk.summary='Introducción al documento sobre el sistema binario Althéra, con índice detallando 10 secciones que abordan desde su descubrimiento hasta aspectos sobre habitabilidad y perspectivas futuras.'
chunk.sections_found=['Índice']
Text:


# Un nuevo y fascinante vecino: Althéra 

## Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

chunk: 1 ----------------
chunk.reason_of_chunk=<ChunkReason.section: 'section'>
chunk.summary='Se presenta la primera detección en 2032 del sistema binario Althéra por observaciones del telescopio James Webb, identificando características en el flujo luminoso y curvas de luz que indican dos estrellas y cuerpos circumbinarios.'
chunk.sections_found=['1. Historia del descubrimiento', '1.1 Primeras observaciones y sospechas iniciales']
Text:


## 1. Historia del descubrimiento

### 1.1 Primeras observaciones y sospechas iniciales

El sistema binario Althéra ( $H D 4579 A B$ ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.

chunk: 2 ----------------
chunk.reason_of_chunk=<ChunkReason.semantic: 'semantic'>
chunk.summary='Confirmación del sistema Althéra y sus características planetarias por un equipo internacional usando técnicas avanzadas como espectroscopía, fotometría y interferometría de radio, confirmando cinco planetas y cinturones de escombros.'
chunk.sections_found=['1.2 Confirmación mediante técnicas combinadas']
Text:


...

### 2.3 Singularidades astronómicas

Lo que hace a Althéra excepcional frente a otros sistemas binarios conocidos es la estabilidad térmica en su zona habitable circumbinaria. Simulaciones realizadas con el Centro de Modelado Planetario de la NASA indican que Aurelia III recibe un flujo de radiación promedio similar al de la Tierra, pero con variaciones estacionales más complejas debido a los ciclos combinados de ambas estrellas.

Entre sus rasgos distintivos:

- Luminiscencia variable combinada: las diferencias de color y brillo entre Althéra A y B generan amaneceres y atardeceres dobles de tonalidades doradas y anaranjadas.
- Influencia gravitacional múltiple: la presencia de dos gigantes gaseosos exteriores, Zephyros IV y Krion V, ayuda a limpiar el espacio interior de objetos errantes, protegiendo a los planetas habitables de impactos masivos frecuentes.
- Cinturón de Arges activo: región rica en asteroides metálicos, con colisiones frecuentes que producen brillantes lluvias de meteoros observables desde Aurelia III.

En conjunto, Althéra constituye un laboratorio natural para estudiar cómo la vida -si existe o llegara a desarrollarse- podría adaptarse a condiciones lumínicas, gravitacionales y climáticas mucho más complejas que las de nuestro sistema solar.

chunk: 8 ----------------
chunk.reason_of_chunk=<ChunkReason.section: 'section'>
chunk.summary='Introducción a las dos estrellas principales de Althéra, con clasificación espectral F7V y K2V, describiendo sus características de temperatura y luz, que crean un ambiente luminoso único para la fotosíntesis en planetas circumbinarios.'
chunk.sections_found=['3. Los soles de Althéra', '3.1 Clasificación espectral y diferencias principales']
Text:


Si quieres, puedo continuar directamente con la Sección 3: Las estrellas centrales, donde detallaríamos propiedades físicas, químicas y de emisión de cada una, incluyendo su papel en la habitabilidad del sistema.

Perfecto, aquí tienes la Sección 3 con un tono técnico pero atractivo, para que las dos estrellas de Althéra cobren protagonismo propio:

# 3. Los soles de Althéra 

### 3.1 Clasificación espectral y diferencias principales

El corazón del sistema Althéra está formado por un par estelar bien contrastado:

- Althéra A (HD 4579 A): estrella de tipo F7V, más caliente y luminosa que el Sol, con un tono blanco-dorado.
- Althéra B (HD 4579 B): estrella de tipo K2V, más fría y longeva, de tonalidad naranja intensa.

Esta combinación de espectros produce un ambiente luminoso único en los planetas circumbinarios, donde la radiación es más equilibrada en el rango visible y cercana al infrarrojo, favoreciendo la fotosíntesis de potenciales formas de vida adaptadas a ambas fuentes.

chunk: 9 ----------------
chunk.reason_of_chunk=<ChunkReason.section: 'section'>
chunk.summary='Comparación de masas, radios y luminosidades entre Althéra A y B, destacando que Althéra A domina la energía total, mientras que Althéra B regula térmicamente para estabilizar las temperaturas en la zona habitable. Incluye imagen representativa.'
chunk.sections_found=['3.2 Masas, radios y luminosidades comparadas']
Text:


### 3.2 Masas, radios y luminosidades comparadas

El sistema binario Althéra está formado por dos estrellas de características bien contrastadas: Althéra A, más masiva y luminosay Althéra B, más pequeña y fría. Aunque la luminosidad de Althéra A domina el aporte energético global, la presencia de Althéra B, O, actúa como un regulador térmico que suaviza las variaciones extremas de temperatura en la zona habitable, contribuyendo a su estabilidad a largo plazo.
![img-0.jpeg](img-0.jpeg)

Figure 1: Ocaso en Althéra

In [58]:
corpus_llm = []

for chunk in chunks.chunks:
    doc_i = Document(
        page_content=chunk.text,
        metadata={"source": "Althera",
                  "sections_found": chunk.sections_found,
                  "reason_of_chunk": chunk.reason_of_chunk.value}
    )
    corpus_llm.append(doc_i)

In [59]:
utils.save_documents(
    corpus_llm,
    path_output /  "llmbased" / "Althera.json"
)
